In [9]:
import easydict
import gc
import pandas as pd

from preprocessor import PairwisePreprocessor, _20CodeCellPreprocessor
from dataset import _20SampleDataset, PairwiseDataset, _20sample_data_setup, pairwise_data_setup
from train import pairwise_train_setup, train_setup
from util import pairwise_debug_setup, _20sample_debug_setup
from metrics import kendall_tau
from train import train

args = {
    'model_name_or_path': 'microsoft/graphcodebert-base',

    'input_path': '../input/',

    'train_path': './data/train.csv',
    'train_mark_path': './data/train_mark.csv',
    'train_features_path': './data/train_fts.json',

    'val_path': "./data/val.csv",
    'val_mark_path': './data/val_mark.csv',
    'val_features_path': './data/val_fts.json',

    'output_path': './output-graphcodebert-20sample-es-cos-dropout-debug',

    'md_max_len': 64,
    'total_max_len': 512,
    'batch_size': 32,
    'accumulation_steps': 1,
    'epoch': 3,
    'epochs': 5,
    'n_workers': 8,
    'debug': False,
    'load_train': False,
    'max_lr': 3e-5,
    'min_lr': .3e-6,
    'kfold': True
}

args = easydict.EasyDict(args)

In [10]:
preprocessor = _20CodeCellPreprocessor(**vars(args))
train_df, val_df, train_df_mark, val_df_mark, train_fts, val_fts = preprocessor.run()

# print('before debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

# kfolds = []
# if args.debug:
#     for i in range(5):
#         fold = _20sample_debug_setup(train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts)
#         kfolds.append(fold)
    
# train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[0]
# print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

# train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[1]
# print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

# train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[2]
# print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

# train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[3]
# print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))

# train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[4]
# print('after debug', train_df.shape, val_df.shape, train_df_mark.shape, val_df.shape, len(train_fts), len(val_fts))


train_df, val_df are already exits
train_fts, val_fts are already exists


In [11]:
df_orders = pd.read_csv(args.input_path + 'train_orders.csv',
                        index_col='id',
                        squeeze=True).str.split()

C:\Users\ansrm\AppData\Local\Temp\ipykernel_13768\2808593356.py:1: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df_orders = pd.read_csv(args.input_path + 'train_orders.csv',


In [13]:
# for i in range(5):
#     train_df, train_df_mark, train_fts, val_df, val_df_mark, val_fts = kfolds[i]
import torch

train_loader, val_loader = _20sample_data_setup(train_df_mark, val_df_mark, train_fts, val_fts, args)

args.num_train_steps = args.epochs * len(train_loader) / args.accumulation_steps

model, optimizer, scheduler, scaler = train_setup(args)
model.load_state_dict(torch.load('./output-graphcodebert-20sample-es-cos-dropout-debug/model_epoch_2_0.8441190203208906.bin'))
model.cuda()

train(model, train_loader, val_loader, optimizer, scheduler, scaler, val_df, df_orders, args)

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to

100%|██████████| 6749/6749 [13:21<00:00,  8.42it/s]
Preds score 0.8397768916698124
Epoch 5 Loss: 0.1134 lr: [2.565053570062023e-05, 2.565053570062023e-05]: 100%|██████████| 60941/60941 [6:21:31<00:00,  2.66it/s]    
100%|██████████| 6749/6749 [13:23<00:00,  8.40it/s]
Preds score 0.8442684377372627
100%|██████████| 6749/6749 [13:19<00:00,  8.44it/s]
Preds score 0.8499189849500974
Epoch 7 Loss: 0.0865 lr: [4.64946429937977e-06, 4.64946429937977e-06]: 100%|██████████| 60941/60941 [6:18:38<00:00,  2.68it/s]      
100%|██████████| 6749/6749 [13:21<00:00,  8.42it/s]
Preds score 0.8526143515753599


OSError: [WinError 433] 없는 장치를 지정했습니다: './output-graphcodebert-20sample-es-cos-dropout-debug'